In [ ]:
import pandas as pd
import os
from plotnine import *

In [ ]:
df = pd.read_excel("Local Inclusionary Housing Policies.xlsx",sheet_name="data",skiprows=1)
df['Government Name'] = df['Government Name'].str.replace("City of","").str.replace("City","").str.strip()
df.head()

In [ ]:
df_2 = df[df['Minimum Project Size'].notna()].copy()
df_2.shape

In [ ]:
df_2['Minimum Project Size'].unique()

In [ ]:
df_3 = df_2[df_2['Minimum Project Size'].str.lower().str.contains("units")].copy()
df_3.head()

In [ ]:
df_pops = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")
df_pops = df_pops[df_pops['Population']>=200000][['City','State','Population']]
df_pops.head()

In [ ]:
df_pops[df_pops['State']=='New Jersey']

In [ ]:
df_merged = df_3.merge(df_pops,how='inner',left_on='Government Name',right_on='City').sort_values("Population",ascending=False)
df_merged[['Program ID','Population','Year Program Adopted','Year Program Updated','Program Applicable Areas','Minimum Project Size','Minimum Set-aside','Incentive','Affordable Housing Compliance Option','Total Affordable Units']]

# Combining with Building Data

In [ ]:
os.listdir("../data/")

In [ ]:
df_nj = pd.read_csv("../data/NJ_Construction_Permit_Data.csv",
                    usecols = ['Municipality Name','MuniType','County',
                                   'Permit Status Description','Permit Date','Certificate Date',
                                   'Permit Type Description','Certificate Type Description',
                                   'Sale Units Gained','Rental Units Gained','Use Group Description'])
df_nj['Units Gained'] = df_nj['Sale Units Gained']+df_nj['Rental Units Gained']
df_nj = df_nj[df_nj['Units Gained']>=2].copy()
print(df_nj.shape)
df_nj.head()

In [ ]:
df_sf = pd.read_csv("../data/SF_Building_Permits.csv",
                   usecols = ['Permit Type Definition','Permit Creation Date',
                             'Current Status',
                              'Current Status Date',
                              'Filed Date','Issued Date','Completed Date','First Construction Document Date',
                             'Existing Units','Proposed Units'])
df_sf = df_sf[(df_sf['Proposed Units']>=3)&(df_sf['Proposed Units']!=df_sf['Existing Units'])].copy()
df_sf['Permit Creation Date'] = pd.to_datetime(df_sf['Permit Creation Date'])
print(df_sf.shape)
df_sf.head()

In [ ]:
df_pd = pd.read_csv("../data/PDX_Residential_Building_Permits.csv",
                   usecols = ['STATUS','YEAR_','NEWCLASS','NEWTYPE','NEW_UNITS','IS_ADU'])
df_pd = df_pd[(df_pd['NEW_UNITS']>=2)&(df_pd['IS_ADU']==False)].drop("IS_ADU",axis=1).copy()
print(df_pd.shape)
df_pd.head()

In [ ]:
df_dc = pd.read_csv("../data/DC_appraisals.csv",usecols=['NUM_UNITS','AYB','YR_RMDL','EYB'])
print(df_dc.shape)
df_dc.head()

In [ ]:
df_cc = pd.read_csv("../data/Cook_appraisals.csv",usecols=['Town Code','Census Tract','Number of Units','Age','Apartments'])
# Age = 10 could mean NA, need to figure out how to distinguish, maybe just drop
df_cc = df_cc[df_cc['Number of Units']>=2].copy()
print(df_cc.shape)
df_cc.head()

In [ ]:
df_md = pd.read_csv("../data/MD_appraisals.csv",usecols=['CM_BLDUNTS','OBJECTID','CT2010'])
df_md_2 = pd.read_csv("../data/MD_appraisal_details.csv",usecols=['OBJECTID','BL_YEARBLT','JURSCODE'])
df_md = df_md[df_md['CM_BLDUNTS']>=2].copy()
df_md = df_md.merge(df_md_2,on='OBJECTID',how='inner')
print(df_md.shape)
df_md.head()

# EDA

In [ ]:
df_nj['Municipality Name'].value_counts().head()

In [ ]:
df_newark = df_nj[df_nj['Municipality Name']=='NEWARK']
df_newark.head()

In [ ]:
df_newark['Units Gained'].value_counts()

In [ ]:
def plot_unit_dist(data,unit_col='Units'):
    counts = pd.DataFrame(data[unit_col].value_counts()).reset_index()
    counts.columns = ['Units','Freq']
    plot = (ggplot(counts,aes(x='Units',y='Freq')) + 
           geom_bar(stat='identity'))
    return(plot)

## SF

In [ ]:
plot_unit_dist(df_sf[(df_sf['Proposed Units']<50)&(df_sf['Proposed Units']>=3)],unit_col='Proposed Units')

In [ ]:
df_sf['Current Status'].value_counts()

In [ ]:
plot_unit_dist(df_sf[(df_sf['Proposed Units']<=40)&(df_sf['Proposed Units']>=5)&(df_sf['Permit Creation Date']>='06/01/2016')&(df_sf['Current Status'].isin(['complete','issued']))],unit_col='Proposed Units')

## Portland

In [ ]:
plot_unit_dist(df_pd.query("NEW_UNITS>=5&NEW_UNITS<=50&YEAR_>=2017"),"NEW_UNITS")

In [ ]:
plot_unit_dist(df_pd.query("NEW_UNITS>=5&NEW_UNITS<=50&YEAR_<2017"),"NEW_UNITS")